In [3]:
import requests
import pandas as pd
import re
import numpy as np
from bs4 import BeautifulSoup

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

response = requests.get('https://www.goodreads.com/list/show/6934.Science_Fiction_Books_by_Female_Authors', headers=headers)
doc = BeautifulSoup(response.text)

In [ ]:
# Rank	1
# Title	The Handmaid's Tale
# Author	Margaret Atwood
# Score	score: 30,733
# Votes	314 people voted
# Rating	4.09 avg rating — 1,101,120 ratings

In [99]:
rows = []
table = doc.find_all(class_='tableList')[0]
books = table.find_all("tr")
for book in books:
    row = {}
    row['rank'] = book.find(class_='number').text.strip()
    row['title'] = book.find(class_='bookTitle').text.strip()
    row['author'] = book.find(class_='authorName').text.strip()
    row['score'] = book.find_all('a')[3].text
    row['votes'] = book.find_all('a')[4].text
    row['rating'] = book.find(class_='uitext').text.strip()
    rows.append(row)
    
df = pd.DataFrame(rows)
df.head()  

,author,rank,rating,score,title,votes
0,Margaret Atwood,1,"4.09 avg rating — 1,101,568 ratings","score: 30,733",The Handmaid's Tale,314 people voted
1,Suzanne Collins,2,"4.33 avg rating — 5,741,857 ratings","score: 28,553","The Hunger Games (The Hunger Games, #1)",292 people voted
2,Mary Wollstonecraft Shelley,3,"3.78 avg rating — 1,023,245 ratings","score: 21,909",Frankenstein,224 people voted
3,Madeleine L'Engle,4,"4.01 avg rating — 902,852 ratings","score: 18,720","A Wrinkle in Time (Time Quintet, #1)",196 people voted
4,Ursula K. Le Guin,5,"4.06 avg rating — 98,748 ratings","score: 17,920",The Left Hand of Darkness (Hainish Cycle #4),184 people voted


In [101]:
df['score'] = df.score.str.replace(",","")
df['score'] = df.score.str.extract(r"score: (.*)")

In [102]:
df.head()

,author,rank,rating,score,title,votes
0,Margaret Atwood,1,"4.09 avg rating — 1,101,568 ratings",30733,The Handmaid's Tale,314 people voted
1,Suzanne Collins,2,"4.33 avg rating — 5,741,857 ratings",28553,"The Hunger Games (The Hunger Games, #1)",292 people voted
2,Mary Wollstonecraft Shelley,3,"3.78 avg rating — 1,023,245 ratings",21909,Frankenstein,224 people voted
3,Madeleine L'Engle,4,"4.01 avg rating — 902,852 ratings",18720,"A Wrinkle in Time (Time Quintet, #1)",196 people voted
4,Ursula K. Le Guin,5,"4.06 avg rating — 98,748 ratings",17920,The Left Hand of Darkness (Hainish Cycle #4),184 people voted


In [117]:
df['no_of_ratings'] = df.rating.str.extract(r"—\s(.*)\s") 

In [119]:
df['rating'] = df.rating.str.extract(r"(\d.\d+)")

In [131]:
df.head()

,author,rank,rating,score,title,votes,no_of_ratings,series,no_in_series
0,Margaret Atwood,1,4.09,30733,The Handmaid's Tale,314 people voted,"1,101,568",NaN,NaN
1,Suzanne Collins,2,4.33,28553,"The Hunger Games (The Hunger Games, #1)",292 people voted,"5,741,857","The Hunger Games, #1",1
2,Mary Wollstonecraft Shelley,3,3.78,21909,Frankenstein,224 people voted,"1,023,245",NaN,NaN
3,Madeleine L'Engle,4,4.01,18720,"A Wrinkle in Time (Time Quintet, #1)",196 people voted,"902,852","Time Quintet, #1",1
4,Ursula K. Le Guin,5,4.06,17920,The Left Hand of Darkness (Hainish Cycle #4),184 people voted,"98,748",Hainish Cycle #4,4


In [136]:
df['series'] = df.series.str.replace(r",? #.*","")

In [126]:
df['no_in_series'] = df.series.str.extract(r"#(.*)")

In [142]:
df['title'] = df.title.str.extract("([^\(]+)")

In [143]:
df.head()

,author,rank,rating,score,title,votes,no_of_ratings,series,no_in_series
0,Margaret Atwood,1,4.09,30733,The Handmaid's Tale,314 people voted,"1,101,568",NaN,NaN
1,Suzanne Collins,2,4.33,28553,The Hunger Games,292 people voted,"5,741,857",The Hunger Games,1
2,Mary Wollstonecraft Shelley,3,3.78,21909,Frankenstein,224 people voted,"1,023,245",NaN,NaN
3,Madeleine L'Engle,4,4.01,18720,A Wrinkle in Time,196 people voted,"902,852",Time Quintet,1
4,Ursula K. Le Guin,5,4.06,17920,The Left Hand of Darkness,184 people voted,"98,748",Hainish Cycle,4


In [144]:
df.to_csv("books.csv",index=False)